In [1]:
import pickle
import os
import pandas as pd
from data_work_depl import transf_units
import numpy as np

path = os.getcwd()
# path_res = os.path.join(path, 'data_out')

#####################################################################
# For the lin.el. example of the paper: 
# RC_nonlin = False
# path_res = os.path.join(path, 'data_out\\data_20241220_1053_casexx')
# path_train = '..\\04_Training'
# path_stats = os.path.join(path_train,'new_data\\_simple_logs\\v_182')
#####################################################################

#####################################################################
# for the glass example
# RC_nonlin = False
# path_res = os.path.join(path, 'data_out\\data_20250207_1154_casexx')
# path_train = '..\\04_Training'
# path_stats = os.path.join(path_train,'new_data\\_simple_logs\\v_198')
#####################################################################

# for the nonlinear RC example
RC_nonlin = False
path_res = os.path.join(path, 'data_out\\data_20260108_1425_casexx\\1400')
path_train = '..\\04_Training'
path_stats = os.path.join(path_train,'new_data\\_simple_logs\\v_474')
path_stats_D = os.path.join(path_train,'new_data\\_simple_logs\\v_474')


# Load data
with open(os.path.join(path_res, 'mat_res_norm.pkl'),'rb') as handle:
		mat_res_pkl = pickle.load(handle)
with open(os.path.join(path_stats, 'mat_data_stats.pkl'),'rb') as handle:
        mat_data_stats = pickle.load(handle)
with open(os.path.join(path_stats, 'mat_data_np_TrainEvalTest.pkl'),'rb') as handle:
		mat_data_np_TrainEvalTest = pickle.load(handle)


if not RC_nonlin:	
	# use these for lin.el. / glass
	# with open(os.path.join(path_res, 'mat_res_NN_sig.pkl'),'rb') as handle:			
	# 		mat_res_pkl_NN = pickle.load(handle)
	pass
if RC_nonlin:
	# use this for nonlinear RC
	with open(os.path.join(path_res, 'mat_res_NN.pkl'),'rb') as handle:					
			mat_res_pkl_NN = pickle.load(handle)
	with open(os.path.join(path_stats_D, 'mat_data_stats.pkl'),'rb') as handle:
			mat_data_stats_D = pickle.load(handle)	
	with open(os.path.join(path_stats_D, 'mat_data_np_TrainEvalTest.pkl'),'rb') as handle:
			mat_data_np_TrainEvalTest_D = pickle.load(handle)


# Transform units of statistics from MN and cm to N and mm
for key, value in mat_data_stats['stats_y_train'].items():
	mat_data_stats['stats_y_train'][key] = transf_units(value[0:8].reshape(1,8), 'sig', forward=False).reshape(8,)
if RC_nonlin:
	for key, value in mat_data_stats_D['stats_y_train'].items():
		value_ = np.zeros((1,8,8))
		value_[:,:6,:6] = value[:36].reshape((-1,6,6))
		value_[:,6,6] = value[36].reshape((-1,1))
		value_[:,7,7] = value[37].reshape((-1,1))
		mat_data_stats_D['stats_y_train'][key] = transf_units(value_, 'D', forward=False, linel=False)


# mat_res_raw = pd.DataFrame.from_dict(mat_res_pkl)
# mat_res_raw_NN = pd.DataFrame.from_dict(mat_res_pkl_NN)
# print(mat_res_raw.shape)
# print(mat_res_raw_NN.shape)

In [3]:
# Note: Within that file, line 0 set the desired column to be displayed in the figure below
# needs to manually be adjusted to correct file... for now set to the "real" simulation, not nn-hybrid one.
%run Plot_Dash_vb2

TypeError: descriptor 'reshape' for 'numpy.ndarray' objects doesn't apply to a 'str' object

In [ ]:
print(mat_res_raw['NODESG'].shape)
print(mat_res_raw_NN['NODESG'].shape)

from main_utils_vb import imshow_plots


imshow_plots(mat_res_raw['COORD'], mat_res_raw['ms'], mat_res_raw['L'])

In [ ]:
# Check whether the plotted points are in the range of the dataset
from data_work_depl import plots_mike_dataset

path = os.getcwd()
path_plots = os.path.join(path, 'plots')
mat_res_raw0 = pd.DataFrame.from_dict(mat_res_pkl)
mat_res_raw0_NN = pd.DataFrame.from_dict(mat_res_pkl_NN)
mat_res_raw_t1 = mat_res_raw0.loc[0,:]
mat_res_raw_NN_t1 = mat_res_raw0_NN.loc[0,:]

# Get simulation data in correct format
eps_sim = mat_res_raw0['eps_g'][0][:,0,0,:]
x_sim = eps_sim
y_sim = mat_res_raw0['sig_g'][0][:,0,0,:]

# Get prediction data in correct format
eps_sim_NN = mat_res_raw0_NN['eps_g'][0][:,0,0,:]
x_sim_NN = eps_sim_NN
y_sim_NN = mat_res_raw0_NN['sig_g'][0][:,0,0,:]

# Get training, evaluation, test data in correct format
x_all_u = np.concatenate((mat_data_np_TrainEvalTest['X_train'], mat_data_np_TrainEvalTest['X_eval'], mat_data_np_TrainEvalTest['X_test']), axis=0)
y_all_u = np.concatenate((mat_data_np_TrainEvalTest['y_train'], mat_data_np_TrainEvalTest['y_eval'], mat_data_np_TrainEvalTest['y_test']), axis=0)
x_all = transf_units(x_all_u, 'eps-t', forward=False)[:,0:8]
y_all = transf_units(y_all_u, 'sig', forward=False)[:,0:8]

# Plot scatter for eps-sig
# plots_mike_dataset(x_all, x_sim, x_sim_NN, y_all, y_sim, y_sim_NN, path_plots, 'sim')


if RC_nonlin:
    # Plot scatter for one row of eps-D_ij
    a = 0   # start index of D
    b = 38   # end index of D
    D_all_u = np.concatenate((mat_data_np_TrainEvalTest_D['y_train'], mat_data_np_TrainEvalTest_D['y_eval'], mat_data_np_TrainEvalTest_D['y_test']), axis=0)
    D_all_u_ = np.zeros((D_all_u.shape[0],8,8))
    D_all_u_[:,:6,:6] = D_all_u[:,:36].reshape((-1,6,6))
    D_all_u_[:,6,6] = D_all_u[:,36].reshape((-1,))
    D_all_u_[:,7,7] = D_all_u[:,37].reshape((-1,))
    D_all_ = transf_units(D_all_u_, 'D', forward=False, linel=False)
    D_sim_ = mat_res_raw0['De_tot'][0][:,0,0,:]
    D_sim_NN_ = mat_res_raw_NN['De_tot'][0][:,0,0,:]
    D_all, D_sim, D_sim_NN = np.zeros((D_all_.shape[0],38)), np.zeros((D_sim_.shape[0],38)), np.zeros((D_sim_NN_.shape[0],38))
    D_all[:,:36], D_sim[:,:36], D_sim_NN[:,:36] = D_all_[:,:6,:6].reshape((-1,36)), D_sim_[:,:6,:6].reshape((-1,36)), D_sim_NN_[:,:6,:6].reshape((-1,36))
    D_all[:,36], D_sim[:,36], D_sim_NN[:,36] = D_all_[:,6,6].reshape((-1,)), D_sim_[:,6,6].reshape((-1,)), D_sim_NN_[:,6,6].reshape((-1,))
    D_all[:,37], D_sim[:,37], D_sim_NN[:,37] = D_all_[:,7,7].reshape((-1,)), D_sim_[:,7,7].reshape((-1,)), D_sim_NN_[:,7,7].reshape((-1,))
    
    plots_mike_dataset(x_all, x_sim, x_sim_NN, D_all[:,a:b], D_sim[:,a:b], D_sim_NN[:,a:b], path_plots, 'sim_D')

In [ ]:
from Plot_NoDash import *

main_plt(mat_res_raw_t1, 'sig', path_plots)
# main_plt(mat_res_raw_NN_t1, 'sig', path_plots, nn = True)
# main_plt(mat_res_raw_t1, 'eps', path_plots)
# main_plt(mat_res_raw_NN_t1, 'eps', path_plots, nn = True)
# main_plt(mat_res_raw_t1, 'u', path_plots)
# main_plt(mat_res_raw_NN_t1, 'u', path_plots, nn = True)

# Delta-plot [there's a bug with this plot...]
# main_plt_delta(mat_res_raw_t1, mat_res_raw_NN_t1, 'dsig', path_plots, type_err = 'nrse', stats = mat_data_stats)
# main_plt_delta(mat_res_raw_t1, mat_res_raw_NN_t1, 'dsig', path_plots, type_err = 'rrse', stats = mat_data_stats)
# main_plt_delta(mat_res_raw_t1, mat_res_raw_NN_t1, 'dsig', path_plots, type_err = 'nrse', stats = mat_data_stats)
# main_plt_delta(mat_res_raw_t1, mat_res_raw_NN_t1, 'deps', path_plots, type_err = 'max', stats = mat_data_stats)
# main_plt_delta(mat_res_raw_t1, mat_res_raw_NN_t1, 'du', path_plots, type_err = 'max', stats = mat_data_stats)

##########################
# Comparison plots (paper)
comp_list_p = ['Nx', 'Ny', 'Nxy', 'Mx', 'My', 'Mxy', 'Qx', 'Qy']
main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'sig', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = comp_list_p, paper=True)

##########################
# Comparison plots (other)
main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'sig', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = ['Ny', 'My', 'Qy'])
main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'eps', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = ['epsy', 'chiy', 'gamy'])
# main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'sig', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = ['My', 'Qy'])
# main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'eps', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = ['chiy', 'gamy'])

# main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'eps', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = ['epsx', 'epsxy'])
# main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'sig', path_plots, type_err = 'nrse', stats = mat_data_stats, comp_list = ['Nx', 'Nxy'])
# main_plt_comp(mat_res_raw_t1, mat_res_raw_NN_t1, 'sig', path_plots, type_err = 'max', stats = mat_data_stats, comp_list = ['Nx', 'Ny', 'Nxy', 'Mx', 'My', 'Mxy', 'Qx', 'Qy'])

# mat_res_raw_t2 = mat_res_raw0.loc[1,:]
# main_plt(mat_res_raw_t2, 'sig', path_plots)
# main_plt(mat_res_raw_t2, 'eps', path_plots)
# main_plt(mat_res_raw_t2, 'u', path_plots)
# main_plt(mat_res_raw, 'geom+load')        # not working

In [ ]:
from datetime import datetime
import shutil

def copy_files_to_plots_folder(src_folder, dest_folder, files_to_copy):
    """
    Copies specified files from src_folder to a new 'plots' folder within dest_folder.
    """
    # Create the 'plots' folder within the destination folder
    plots_folder = os.path.join(dest_folder, 'plots')
    os.makedirs(plots_folder, exist_ok=True)
    
    # Copy only the specified files to the 'plots' folder
    for file_name in files_to_copy:
        src_path = os.path.join(src_folder, file_name)
        dest_path = os.path.join(plots_folder, file_name)
        
        if os.path.exists(src_path):  # Ensure the file exists in the source folder
            shutil.copy2(src_path, dest_path)
        else:
            print(f"{file_name} does not exist in the source folder.")
    
    print(f"Specified files copied to {plots_folder}")

In [ ]:
save_folder = False
filenames = ['single_sig_NLFEA.png', 'single_sig_NN.png', 'single_eps_NLFEA.png', 'data_scatter_sim.png', 'comparison_[\'Nx\', \'My\', \'Qy\'].png']

if save_folder:
    src_folder = os.path.join(os.getcwd(), 'plots')
    dest_folder = path_res
    copy_files_to_plots_folder(src_folder, dest_folder, filenames)


In [ ]:
#### PLOTS NOT IN USE ###

In [ ]:
# from Data_Plotting import statistics
# # print(mat_res_raw.keys())
# path_plt = os.path.join(path, 'plots\output')
# n_bins = 20

# statistics(mat_res_raw.loc[:,'sig_g'], 'n_x', 'sig_g', n_bins, path_plt, single = True)
# statistics(mat_res_raw.loc[:,'sig_g'], 'm_xy', 'sig_g', n_bins, path_plt, single = True)
# statistics(mat_res_raw.loc[:,'sig_g'], 'n_x', 'sig_g', n_bins, path_plt, single = False)
# statistics(mat_res_raw.loc[:,'eps_g'], 'eps_x', 'eps_g', n_bins, path_plt, single = False)

In [ ]:
# Try to implement Andreas' plotting function

# from Plot_Dash_vb import plot_an                # (this is sadly not working, maybe try again later)
# from Plot_Dash_vb import update_graph, make_layout
# from maindash import app
# import dash
# from dash import dcc
# from dash import html
# from dash.dependencies import Input, Output

# plotter = plot_an(mat_res_raw.loc[0,:])

# if __name__== '__main__': 
#     app.layout = make_layout(plotter)
#     # update_graph()
#     app.run_server(debug = True)

In [ ]:
# my own plotting function for single plots (do not use)

# from Data_Plotting import single_analysis 
# num_ele = np.array([2,2])
# single = True

# single_analysis(mat_res_raw.loc[0,:],'sig_g', 'n_x', num_ele, single)

In [ ]:
# Random tests


# import matplotlib.pyplot as plt

# from mpl_toolkits.mplot3d import axes3d

# plt.style.use('_mpl-gallery')

# # Make data
# X, Y, Z = axes3d.get_test_data(0.05)

# # Plot
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# ax.plot_wireframe(X, Y, Z, rstride=10, cstride=10)

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

# plt.show()